# DEBUG OUTPUT from Arduino

In [1]:
otp_data_read_0 = ["01","34","74"]
otp_data_read_1 = ["01","E1","85"]
otp_data_read_2 = ["02","AB","4E"]
otp_data_read_3 = ["0E","F0","6E"]
c1,c2,c3,c4 = 308.00,481.00,683.00,3824.00
rawData = ["AE","69","20","49","00","CB","6B","C6","2F"]
press_raw = 11430217
temp_raw = 27590
p_LUT = [3670508.25,7832320.50,12059716.00]
A,B,C,t = 247159.23,-4018263293952.00,16206216.00,-5178.00
press = 1017.6191

# functions implementation in Python

In [2]:
def parse_otp_data(otp_data_read):
    print("data_read: 0x"+"".join(otp_data_read))
    otp_data = otp_data_read[0]+otp_data_read[1]
    print("data: 0x" + otp_data)
    val = int(otp_data, 16)
    print("16-bit big endian signed integer: " + str(val))
    return val

In [3]:
def parse_raw_data(raw_data):
    press_data = raw_data[0] + raw_data[1] + raw_data[3]
    temp_data = raw_data[6] + raw_data[7]
    press_val = int(press_data, 16)
    temp_val = int(temp_data, 16)
    print("RawData: "+",".join(raw_data))
    print("Pressure_data: 0x" + press_data)
    print("Temp_data: 0x" + temp_data)
    print("Pressure_val: {}".format(press_val))
    print("Temp_val: {}".format(temp_val))
    return (press_val, temp_val)

In [4]:
class Invensense_Pressure_Conversion:
    """ Class for conversion of the pressure and temperature output of the Invensense sensor"""
    def __init__(self, sensor_constants): 
        """ 
        Initialize customer formula
        Arguments:
        sensor_constants -- list of 4 integers: [c1, c2, c3, c4] 
        """
        self.sensor_constants = sensor_constants
        # configuration for Pressure Samples 
        self.p_Pa_calib = [45000.0, 80000.0, 105000.0] 
        self.LUT_lower = 3.5 * (2**20)
        self.LUT_upper = 11.5 * (2**20) 
        self.quadr_factor = 1 / 16777216.0 
        self.offst_factor = 2048.0
        
    def calculate_conversion_constants(self, p_Pa, p_LUT): 
        """
        calculate temperature dependent constants
        Arguments:
        p_Pa -- List of 3 values corresponding to applied pressure in Pa
        p_LUT -- List of 3 values corresponding to the measured p_LUT values at the applied pressures. 
        """
        C = (p_LUT[0] * p_LUT[1] * (p_Pa[0] - p_Pa[1]) + p_LUT[1] * p_LUT[2] * (p_Pa[1] - p_Pa[2]) + p_LUT[2] * p_LUT[0] * (p_Pa[2] - p_Pa[0])) / \
        (p_LUT[2] * (p_Pa[0] - p_Pa[1]) + p_LUT[0] * (p_Pa[1] - p_Pa[2]) + p_LUT[1] * (p_Pa[2] - p_Pa[0]))
        A = (p_Pa[0] * p_LUT[0] - p_Pa[1] * p_LUT[1] - (p_Pa[1] - p_Pa[0]) * C) / (p_LUT[0] - p_LUT[1]) 
        B = (p_Pa[0] - A) * (p_LUT[0] + C)
        return [A, B, C]

    def get_pressure(self, p_LSB, T_LSB):
        """ 
        Convert an output from a calibrated sensor to a pressure in Pa.
        Arguments:
        p_LSB -- Raw pressure data from sensor
        T_LSB -- Raw temperature data from sensor
        """
        t = T_LSB - 32768.0
        s1 = self.LUT_lower + float(self.sensor_constants[0] * t * t) * self.quadr_factor
        s2 = self.offst_factor * self.sensor_constants[3] + float(self.sensor_constants[1] * t * t) * self.quadr_factor 
        s3 = self.LUT_upper + float(self.sensor_constants[2] * t * t) * self.quadr_factor
        A, B, C = self.calculate_conversion_constants(self.p_Pa_calib, [s1, s2, s3])
        return A + B / (C + p_LSB)



In [5]:
# taken from C:\eMD-SmartMotion_ICM20789_20689_1.0.0\EMD-App\src\ICM20789-DMP\Invpres.c

INVPRES_CRC8_INIT = 0xFF
INVPRES_RESP_DWORD_LEN = 2
INVPRES_CRC8_POLYNOM = 0x31

def invpres_check_crc(frame):
    crc = INVPRES_CRC8_INIT
    for current_byte in range(INVPRES_RESP_DWORD_LEN):
        crc ^= frame[current_byte]
        for bit in range(8,0,-1):
            if crc & 0x80:
                crc = (crc << 1) ^ INVPRES_CRC8_POLYNOM
            else:
                crc = (crc << 1)
    crc = crc & 0xFF # take only last byte
    return crc


In [6]:
def check_crc(data_array):
    """
    data_array: a list of three hex strings e.g. ['BE', 'EF', '92']
    calculates crc based on the first and second elements.
    return True if CRC matches the third element
    """
    val0 = int(data_array[0], 16)
    val1 = int(data_array[1], 16)
    crc = invpres_check_crc((val0, val1))
    if crc == int(data_array[2], 16):
        return True
    else:
        return False

# Check CRC if raw reading values are correct

In [7]:
check_crc(otp_data_read_0)

True

In [8]:
check_crc(otp_data_read_1)

True

In [9]:
check_crc(otp_data_read_2)

True

In [10]:
check_crc(otp_data_read_3)

True

In [11]:
print(check_crc(rawData[0:3]))
print(check_crc(rawData[3:6]))
print(check_crc(rawData[6:9]))

True
True
True


# Now let's check the calculation

In [12]:
c1p = parse_otp_data(otp_data_read_0)
c2p = parse_otp_data(otp_data_read_1)
c3p = parse_otp_data(otp_data_read_2)
c4p = parse_otp_data(otp_data_read_3)
print("Arduino: c1,c2,c3,c4: {},{},{},{}".format(c1, c2, c3, c4))
print("Python: c1,c2,c3,c4: {},{},{},{}".format(c1p, c2p, c3p, c4p))

data_read: 0x013474
data: 0x0134
16-bit big endian signed integer: 308
data_read: 0x01E185
data: 0x01E1
16-bit big endian signed integer: 481
data_read: 0x02AB4E
data: 0x02AB
16-bit big endian signed integer: 683
data_read: 0x0EF06E
data: 0x0EF0
16-bit big endian signed integer: 3824
Arduino: c1,c2,c3,c4: 308.0,481.0,683.0,3824.0
Python: c1,c2,c3,c4: 308,481,683,3824


In [13]:
p, T = parse_raw_data(rawData)
print("Arduino: press_raw, temp_raw: {},{}".format(press_raw, temp_raw))
print("Python: press_raw, temp_raw: {},{}".format(p, T))

RawData: AE,69,20,49,00,CB,6B,C6,2F
Pressure_data: 0xAE6949
Temp_data: 0x6BC6
Pressure_val: 11430217
Temp_val: 27590
Arduino: press_raw, temp_raw: 11430217,27590
Python: press_raw, temp_raw: 11430217,27590


In [14]:
conversion = Invensense_Pressure_Conversion([c1p, c2p, c3p, c4p])
pressure = conversion.get_pressure(p, T)
print("Arduino: pressure: {}".format(press))
print("Python: pressure: {}".format(pressure/100.))

Arduino: pressure: 1017.6191
Python: pressure: 1017.6191687645006


### YES!!!